In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


In [4]:
df = pd.read_csv("/content/engine_data.csv")

In [5]:
df

,Engine rpm,Lub oil pressure,Fuel pressure,Coolant pressure,lub oil temp,Coolant temp,Engine Condition
0,700,2.493592,11.790927,3.178981,84.144163,81.632187,1
1,876,2.941606,16.193866,2.464504,77.640934,82.445724,0
2,520,2.961746,6.553147,1.064347,77.752266,79.645777,1
3,473,3.707835,19.510172,3.727455,74.129907,71.774629,1
4,619,5.672919,15.738871,2.052251,78.396989,87.000225,0
...,...,...,...,...,...,...,...
19530,902,4.117296,4.981360,4.346564,75.951627,87.925087,1
19531,694,4.817720,10.866701,6.186689,75.281430,74.928459,1
19532,684,2.673344,4.927376,1.903572,76.844940,86.337345,1
19533,696,3.094163,8.291816,1.221729,77.179693,73.624396,1


In [6]:
X = df[['Engine rpm', 'Lub oil pressure', 'Fuel pressure', 'Coolant pressure', 'lub oil temp', 'Coolant temp']]
Y = df['Engine Condition']

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state = 42, test_size = 0.3)

x_train.shape, x_test.shape

((13674, 6), (5861, 6))

In [15]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

In [16]:
x_train_transform = ss.fit_transform(x_train)
x_test_transform = ss.transform(x_test)

In [17]:
x_train_transform

array([[ 0.10385992,  0.30078693, -1.1296357 , -0.0535518 , -0.74102428,
        -0.55233729],
       [-1.14854337, -0.46001281,  0.01505529,  3.80471488, -0.778576  ,
        -1.11496634],
       [ 0.01067516,  0.34837814, -0.19886502,  2.20887769, -0.43884239,
        -1.56858212],
       ...,
       [-0.65652779,  0.00981206, -1.03232067,  3.51039777, -0.56574262,
        -1.00274467],
       [ 1.38235496, -0.55460454, -1.06287575,  0.58635114, -0.27826555,
         0.46610614],
       [-0.36206392, -0.7660318 , -0.09937514,  0.33404566, -0.03047272,
        -1.17297767]])

In [18]:
x_train_t = torch.FloatTensor(np.array(x_train_transform))
y_train_t = torch.FloatTensor(np.array(y_train))
x_test_t = torch.FloatTensor(np.array(x_test_transform))
y_test_t = torch.FloatTensor(np.array(y_test))

In [19]:
class ClassificationModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(in_features = 6, out_features = 120)
        self.act1 = nn.ReLU()
        
        self.layer_2 = nn.Linear(in_features = 120, out_features = 120)
        self.act2 = nn.ReLU()
        
        self.layer_3 = nn.Linear(in_features = 120, out_features = 120)
        self.act3 = nn.ReLU()
        
        self.layer_4 = nn.Linear(in_features = 120, out_features = 120)
        self.act4 = nn.ReLU()
        
        self.layer_5 = nn.Linear(in_features = 120, out_features = 1)
        self.act5 = nn.Sigmoid()
        
    def forward(self, x:torch.tensor) -> torch.tensor:
        x = self.layer_1(x)
        x = self.act1(x)
        
        x = self.layer_2(x)
        x = self.act2(x)
        
        x = self.layer_3(x)
        x = self.act3(x)
        
        x = self.layer_4(x)
        x = self.act4(x)
        
        x = self.layer_5(x)
        output = self.act5(x)
        
        return output

In [20]:
model = ClassificationModel()

model

ClassificationModel(
  (layer_1): Linear(in_features=6, out_features=120, bias=True)
  (act1): ReLU()
  (layer_2): Linear(in_features=120, out_features=120, bias=True)
  (act2): ReLU()
  (layer_3): Linear(in_features=120, out_features=120, bias=True)
  (act3): ReLU()
  (layer_4): Linear(in_features=120, out_features=120, bias=True)
  (act4): ReLU()
  (layer_5): Linear(in_features=120, out_features=1, bias=True)
  (act5): Sigmoid()
)

In [21]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.0543, -0.3515,  0.3917,  0.0041,  0.0620, -0.0126],
         [ 0.1785, -0.3067, -0.0818, -0.2769, -0.0597,  0.0091],
         [-0.0937,  0.0509, -0.2802,  0.2454,  0.2690, -0.0809],
         [ 0.2154, -0.0602,  0.0826, -0.0581, -0.0940,  0.4061],
         [-0.0030, -0.3082,  0.1478,  0.2686,  0.1796,  0.3717],
         [-0.1368,  0.3564, -0.1966,  0.3991, -0.1393,  0.0886],
         [ 0.2657,  0.1896,  0.3870, -0.0522,  0.3084,  0.1372],
         [ 0.2411,  0.3162, -0.1872,  0.0094,  0.3371,  0.2525],
         [-0.0707, -0.3164, -0.1383,  0.2151, -0.1946, -0.2655],
         [-0.0738, -0.1625,  0.0751,  0.2058,  0.3469, -0.3627],
         [-0.2096,  0.3945,  0.2876, -0.1115,  0.0094, -0.2032],
         [-0.1933, -0.3823, -0.0830, -0.2780, -0.1904, -0.3915],
         [-0.1987,  0.3420,  0.0209,  0.0133,  0.2229,  0.2624],
         [ 0.0476,  0.2007, -0.1715, -0.3536,  0.0199, -0.1314],
         [ 0.1971,  0.3510, -0.0326,  0.3000,  0.0895,  0.1181],
  

In [22]:
model.state_dict()

OrderedDict([('layer_1.weight',
              tensor([[-0.0543, -0.3515,  0.3917,  0.0041,  0.0620, -0.0126],
                      [ 0.1785, -0.3067, -0.0818, -0.2769, -0.0597,  0.0091],
                      [-0.0937,  0.0509, -0.2802,  0.2454,  0.2690, -0.0809],
                      [ 0.2154, -0.0602,  0.0826, -0.0581, -0.0940,  0.4061],
                      [-0.0030, -0.3082,  0.1478,  0.2686,  0.1796,  0.3717],
                      [-0.1368,  0.3564, -0.1966,  0.3991, -0.1393,  0.0886],
                      [ 0.2657,  0.1896,  0.3870, -0.0522,  0.3084,  0.1372],
                      [ 0.2411,  0.3162, -0.1872,  0.0094,  0.3371,  0.2525],
                      [-0.0707, -0.3164, -0.1383,  0.2151, -0.1946, -0.2655],
                      [-0.0738, -0.1625,  0.0751,  0.2058,  0.3469, -0.3627],
                      [-0.2096,  0.3945,  0.2876, -0.1115,  0.0094, -0.2032],
                      [-0.1933, -0.3823, -0.0830, -0.2780, -0.1904, -0.3915],
                      [-0.1987, 

In [23]:
loss_fn = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(params = model.parameters(), lr = 0.001)

x_train_t.shape

In [25]:
y_p = model.forward(x_train_t)
y_p.shape

torch.Size([13674, 1])

In [26]:
y_train_t

tensor([0., 1., 1.,  ..., 0., 0., 1.])

In [27]:
epochs = 100

epoch_count = []
train_loss = []
test_loss = []

for epoch in range(epochs):
    
    # Training Loop
    
    optimizer.zero_grad()
    
    model.train()
    
    y_pred = model(x_train_t.to(device))
    
    loss = loss_fn(y_pred.squeeze(dim = 1), y_train_t)
    train_loss.append(loss)
    
    loss.backward()
    
    optimizer.step()
    
    # Testing Loop
    
    model.eval()
    
    y_pred_t = model(x_test_t)
    
    loss1 = loss_fn(y_pred_t.squeeze(dim = 1), y_test_t)
    test_loss.append(loss1)
    
    if epoch % 10 == 0:
            epoch_count.append(epoch)
            train_loss.append(loss.detach().numpy())
            test_loss.append(loss1.detach().numpy())
            print(f"Epoch: {epoch} | Train Loss: {loss} | Test Loss: {loss1} ")
            print('\n')

Epoch: 0 | Train Loss: 0.6570021510124207 | Test Loss: 0.662527322769165 


Epoch: 10 | Train Loss: 0.6454123854637146 | Test Loss: 0.6499459147453308 


Epoch: 20 | Train Loss: 0.6188991665840149 | Test Loss: 0.6258499622344971 


Epoch: 30 | Train Loss: 0.6141247153282166 | Test Loss: 0.6227551102638245 


Epoch: 40 | Train Loss: 0.6128643155097961 | Test Loss: 0.6224855780601501 


Epoch: 50 | Train Loss: 0.6114831566810608 | Test Loss: 0.6222793459892273 


Epoch: 60 | Train Loss: 0.6105238795280457 | Test Loss: 0.62223219871521 


Epoch: 70 | Train Loss: 0.6094542741775513 | Test Loss: 0.6219571828842163 


Epoch: 80 | Train Loss: 0.6083422899246216 | Test Loss: 0.6218470335006714 


Epoch: 90 | Train Loss: 0.6070938110351562 | Test Loss: 0.6228499412536621 




In [28]:
from sklearn.metrics import accuracy_score

with torch.inference_mode():
    y_preds = model(x_test_t)

threshold = 0.0 
y_preds_n = torch.gt(y_preds, threshold)
print("Accuracy of the Trained and Tested ANN for the Binary Classification Model: {}".format(accuracy_score(y_preds_n, y_test_t) * 100))

Accuracy of the Trained and Tested ANN for the Binary Classification Model: 62.22487630097253


In [29]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_preds_n, y_test_t)
cm

array([[   0,    0],
       [2214, 3647]])